# PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

### Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [1]:
import sqlite3
import pandas as pd

con = sqlite3.connect("sqlite_db_pythonsqlite.db")
#cursor = connection.cursor()

In [2]:


Q10 = pd.read_sql_query("""
                            SELECT Facilities.name,
                                    sum(CASE WHEN Bookings.memid = 0 THEN Facilities.guestcost * Bookings.slots
                                    ELSE Facilities.membercost * Bookings.slots END) AS revenue
                            FROM Members, Facilities, Bookings
                            WHERE Bookings.facid = Facilities.facid
                                AND Members.memid = Bookings.memid
                            GROUP BY Facilities.name
                            HAVING revenue < 1000
                            ORDER BY revenue desc""", con)
Q10

,name,revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [3]:
Q11 = pd.read_sql_query("""
                            SELECT 
                                (M.SURNAME || ',' || M.FIRSTNAME) AS MEMBER,
                                (R.SURNAME || ',' || R.FIRSTNAME) AS RECOMMENDED_BY
                            FROM Members as M
                            JOIN Members as R
                            ON M.RECOMMENDEDBY = R.MEMID
                            ORDER BY MEMBER;""", con)
Q11

,MEMBER,RECOMMENDED_BY
0,"Bader,Florence","Stibbons,Ponder"
1,"Baker,Anne","Stibbons,Ponder"
2,"Baker,Timothy","Farrell,Jemima"
3,"Boothe,Tim","Rownam,Tim"
4,"Butters,Gerald","Smith,Darren"
5,"Coplin,Joan","Baker,Timothy"
6,"Crumpet,Erica","Smith,Tracy"
7,"Dare,Nancy","Joplette,Janice"
8,"Genting,Matthew","Butters,Gerald"
9,"Hunt,John","Purview,Millicent"


### Q12: Find the facilities with their usage by member, but not guests

In [4]:
Q12 = pd.read_sql_query("""
                            SELECT
                                F.Name FACILITY_NAME,
                                M.SURNAME || ',' || M.FIRSTNAME MEMBER_NAME,
                                COUNT(B.BOOKID) TOTAL_MEMBER_USAGE
                            FROM Members M
                            INNER JOIN Bookings as B
                            ON B.MEMID = M.MEMID
                            INNER JOIN Facilities as F
                            ON F.FACID = B.FACID
                            WHERE M.MEMID!=0
                            GROUP BY FACILITY_NAME, MEMBER_NAME;""", con)
Q12

,FACILITY_NAME,MEMBER_NAME,TOTAL_MEMBER_USAGE
0,Badminton Court,"Bader,Florence",9
1,Badminton Court,"Baker,Anne",10
2,Badminton Court,"Baker,Timothy",7
3,Badminton Court,"Boothe,Tim",12
4,Badminton Court,"Butters,Gerald",20
...,...,...,...
197,Tennis Court 2,"Smith,Darren",19
198,Tennis Court 2,"Smith,Jack",1
199,Tennis Court 2,"Smith,Tracy",2
200,Tennis Court 2,"Stibbons,Ponder",31


### Q13: Find the facilities usage by month, but not guests 

In [5]:
Q13 = pd.read_sql_query("""
                            SELECT
                                F.Name FACILITY_NAME,
                                strftime('%m', B.STARTTIME) BOOKING_MONTH,
                                COUNT(B.BOOKID) TOTAL_MEMBER_USAGE
                            FROM Members M
                            INNER JOIN Bookings as B
                            ON B.MEMID = M.MEMID
                            INNER JOIN Facilities as F
                            ON F.FACID = B.FACID
                            WHERE M.MEMID!=0
                            GROUP BY FACILITY_NAME, BOOKING_MONTH;""", con)
Q13

,FACILITY_NAME,BOOKING_MONTH,TOTAL_MEMBER_USAGE
0,Badminton Court,07,51
1,Badminton Court,08,132
2,Badminton Court,09,161
3,Massage Room 1,07,77
4,Massage Room 1,08,153
5,Massage Room 1,09,191
6,Massage Room 2,07,4
7,Massage Room 2,08,9
8,Massage Room 2,09,14
9,Pool Table,07,103


In [6]:
#When complete with the database, close the connection.
con.close()